# Requirements

In [1]:
!pip3 install onnxruntime-gpu==1.15.1 onnx transformers===4.30.1 optimum[onnxruntime-gpu]==1.9.1 protobuf==3.20.1 evaluate datasets onnxoptimizer optimum[exporters] diffusers

  Obtaining dependency information for transformers===4.30.1 from https://files.pythonhosted.org/packages/b8/df/b01b5e67cde3883757c9212455cbb9169385dcab5858b7172199126b756d/transformers-4.30.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 2.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers[sentencepiece] to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for transformers[sentencepiece]>=4.26.0 from https://files.pythonhosted.org/packages/21/02/ae8e595f45b6c8edee07913892b3b41f5f5f273962ad98851dc6a564bbb9/transformers-4.31.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.2 MB/s eta 0:00:00
  Obtaining dependency information for transformers[sentencepiece]>=4.26.0 from https://files.pythonhosted.org/packages/5b/0b/e45d26ccd28568013523e04f325432ea88a442b4e3020b757cf4361f0120/transformers-4.30.2-py3-none-any.wh

In [2]:
!nvidia-smi

Sat Jul 22 12:19:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Loading model

As a starting point, we will need to select which model to use, in our case, we will be working with a pre-trained dataset, `naver-clova-ix/donut-base-finetuned-cord-v2`

The next step is to convert our model to ONNX. To do this we can take advantage of the [optimum](https://huggingface.co/docs/optimum/) library from 🤗 Hugging Face, which helps us make our models more efficient in training and inference. In our case, we will take advantage of the implementations for ORT (ONNX Runtime).


In [3]:
from pathlib import Path

model_path = Path("./models/onnx")
!python3 -m optimum.exporters.onnx --model=naver-clova-ix/donut-base-finetuned-cord-v2 --task=vision2seq-lm  ./models/onnx --atol 1e-3 --opset=13

2023-07-22 12:19:05.944357: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Framework not specified. Using pt to export to ONNX.
The task `image-to-text` was manually specified, and past key values will not be reused in the decoding. if needed, please pass `--task image-to-text-with-past` to export using the past key values.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.10/dist-packages/transformers/models/donut/feature_extraction_donut.py:28: FutureWarning: The class DonutFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DonutImageProcessor instead.
  warnings.warn(
Using framework PyTorch: 2.0.1+cu117
/usr/local/lib/python3.10/dist-packages/transformers/models/donut/modeling_donut_swin.py:229: TracerWarning: Converting a tensor to a Python boolean m

## Benchmark

Now, we measure the size and throughput of the models obtained after optimization step.

In [4]:
!ls -lh --block-size=M ./models/onnx

total 1012M
-rw-r--r-- 1 root root   1M Jul 22 12:19 added_tokens.json
-rw-r--r-- 1 root root   1M Jul 22 12:19 config.json
-rw-r--r-- 1 root root 710M Jul 22 12:21 decoder_model.onnx
-rw-r--r-- 1 root root 298M Jul 22 12:20 encoder_model.onnx
-rw-r--r-- 1 root root   1M Jul 22 12:19 generation_config.json
-rw-r--r-- 1 root root   1M Jul 22 12:19 preprocessor_config.json
-rw-r--r-- 1 root root   2M Jul 22 12:19 sentencepiece.bpe.model
-rw-r--r-- 1 root root   1M Jul 22 12:19 special_tokens_map.json
-rw-r--r-- 1 root root   1M Jul 22 12:19 tokenizer_config.json
-rw-r--r-- 1 root root   4M Jul 22 12:19 tokenizer.json


## Optimize by Speedster

In [5]:
!pip3 install protobuf==3.20.1 torch==1.13.1 nncf tensorrt==8.6.1
!pip3 install onnx-graphsurgeon --extra-index-url https://pypi.ngc.nvidia.com
!pip3 install speedster

  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch-tensorrt 1.4.0 requires torch<2.1,>=2.0.1, but you have torch 1.13.1 which is incompatible.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
!python3 -m nebullvm.installers.auto_installer --frameworks all --extra-backends all --compilers all

2023-07-22 12:22:14.575782: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-22 12:22:15.554323: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-22 12:22:17.285714: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-22 12:22:17.292958: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [7]:
import os
import json
from pathlib import Path
import shutil
import re
import transformers
from PIL import Image
from transformers import DonutProcessor, VisionEncoderDecoderModel
import torch
import random
import numpy as np
from datasets import load_dataset
from PIL import Image
import time

model_base_id = "naver-clova-ix/donut-base-finetuned-cord-v2"
device = "cuda"

processor = DonutProcessor.from_pretrained(model_base_id)
model = VisionEncoderDecoderModel.from_pretrained(model_base_id)

# Move model to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
           

In [8]:
import os
import time
from typing import Optional, Tuple

import onnxoptimizer
import onnxruntime
import torch
from PIL import Image
import onnx


import onnxruntime as onnxrt
import requests
from transformers import AutoConfig, AutoModelForVision2Seq, DonutProcessor, VisionEncoderDecoderModel
from transformers.generation.utils import GenerationMixin
from transformers.modeling_outputs import BaseModelOutput, Seq2SeqLMOutput

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
device_desc = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_desc)

class ORTEncoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.main_input_name = "pixel_values"
        self._device = device
        self.session = onnxrt.InferenceSession(
            "./models/onnx/encoder_model.onnx", providers=["CUDAExecutionProvider","CPUExecutionProvider"]
        )
        self.input_names = {input_key.name: idx for idx, input_key in enumerate(self.session.get_inputs())}
        self.output_names = {output_key.name: idx for idx, output_key in enumerate(self.session.get_outputs())}

    def forward(
        self,
        pixel_values: torch.FloatTensor,
        **kwargs,
    ) -> BaseModelOutput:

        onnx_inputs = {"pixel_values": pixel_values.cpu().detach().numpy()}

        # Run inference
        outputs = self.session.run(None, onnx_inputs)
        last_hidden_state = torch.from_numpy(outputs[self.output_names["last_hidden_state"]]).to(self._device)

        return BaseModelOutput(last_hidden_state=last_hidden_state)


class ORTDecoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.main_input_name = "pixel_values"
        self._device = device
        self.session = onnxrt.InferenceSession(
          "./models/onnx/decoder_model.onnx", providers=["CUDAExecutionProvider","CPUExecutionProvider"]
        )
        self.input_names = {input_key.name: idx for idx, input_key in enumerate(self.session.get_inputs())}
        self.output_names = {output_key.name: idx for idx, output_key in enumerate(self.session.get_outputs())}

    def forward(
        self,
        input_ids: torch.LongTensor,
        attention_mask: torch.LongTensor,
        encoder_hidden_states: torch.FloatTensor,
    ) -> Seq2SeqLMOutput:

        onnx_inputs = {
            "input_ids": input_ids.cpu().detach().numpy(),
        }

        if "attention_mask" in self.input_names:
            onnx_inputs["attention_mask"] = attention_mask.cpu().detach().numpy()

        # Add the encoder_hidden_states inputs when needed
        if "encoder_hidden_states" in self.input_names:
            onnx_inputs["encoder_hidden_states"] = encoder_hidden_states.cpu().detach().numpy()

        # Run inference
        outputs = self.session.run(None, onnx_inputs)

        logits = torch.from_numpy(outputs[self.output_names["logits"]]).to(self._device)
        return Seq2SeqLMOutput(logits=logits)

    def prepare_inputs_for_generation(self, input_ids, attention_mask=None, encoder_hidden_states=None, **kwargs):
        if attention_mask is None:
            attention_mask = input_ids.new_ones(input_ids.shape)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "encoder_hidden_states": encoder_hidden_states,
        }


class ORTModelForVision2Seq(VisionEncoderDecoderModel, GenerationMixin):
    def __init__(self, *args, **kwargs):
        config = AutoConfig.from_pretrained(model_base_id)
        super().__init__(config)
        self._device = device

        self.encoder = ORTEncoder()
        self.decoder = ORTDecoder()

    def forward(
        self,
        pixel_values: Optional[torch.FloatTensor] = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        encoder_outputs: Optional[Tuple[Tuple[torch.Tensor]]] = None,
        **kwargs,
    ) -> Seq2SeqLMOutput:
        if encoder_outputs is None:
            encoder_outputs = self.encoder(pixel_values=pixel_values.to(device))

        # Decode
        decoder_attention_mask = decoder_input_ids.new_ones(decoder_input_ids.shape)
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            attention_mask=decoder_attention_mask,
            encoder_hidden_states=encoder_outputs.last_hidden_state,
        )

        return Seq2SeqLMOutput(
            logits=decoder_outputs.logits,
        )

    def prepare_inputs_for_generation(self, input_ids, attention_mask=None, encoder_outputs=None, **kwargs):

        return {
            "decoder_input_ids": input_ids,
            "decoder_atttention_mask": input_ids,
            "encoder_outputs": encoder_outputs,
        }

    @property
    def device(self) -> torch.device:
        return self._device

    @device.setter
    def device(self, value: torch.device):
        self._device = value

    def to(self, device):
        self.device = device
        return self

In [9]:
def read_and_crop(im, device):
  im = Image.open(r"receipt.jpg")
  im = im.convert("RGB")
  newsize = (300, 400)
  im = im.resize(newsize)
  pixel_values = processor(im, return_tensors="pt").pixel_values
  pixel_values=pixel_values.to(device)
  return pixel_values

In [10]:
import numpy as np
from speedster import optimize_model
from speedster import save_model
from torch.utils.data import DataLoader
import onnxruntime as onnxrt
import torch
import os
from PIL import Image
import requests
from timeit import default_timer as timer

# Images
imgs = ['https://media.snopes.com/2017/07/walmart-jajket.jpg']  # batch of images
image = "receipt.jpg"
Image.open(requests.get(imgs[0], stream=True).raw).save(image)

# input data from image
encoded_inputs = [((read_and_crop(image,device),), None) for _ in range(2)]

onnx_model = ORTModelForVision2Seq()
onnx_model = onnx_model.to(device)

# Testing ONNX Inference - START
model.config.decoder_start_token_id = 2
onnx_model.config.vocab_size = onnx_model.config.decoder.vocab_size
onnx_model.config.pad_token_id = onnx_model.config.decoder.pad_token_id = processor.tokenizer.pad_token_id
onnx_model.config.eos_token_id = onnx_model.config.decoder.eos_token_id = processor.tokenizer.sep_token_id

start = timer()

sequence = model.generate(read_and_crop(image,device).to(device),early_stopping=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=False,
    num_beams=1,
    bad_words_ids=[[processor.tokenizer.unk_token_id]])

sequence = processor.batch_decode(sequence, skip_special_tokens=False, device=device)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
print("Generated JSON Output : ",processor.token2json(sequence))

end = timer()
print('elapsed time: ', end - start) # Time in seconds, e.g. 5.38091952400282
# Testing ONNX Inference - END


# Run Speedster optimization
optimized_model = optimize_model(
    model=onnx_model,
    input_data=encoded_inputs,
    optimization_time="unconstrained",
    device="gpu:0",
    metric_drop_ths=0.8
)

save_model(optimized_model, "./models/speedster")


Post-training Optimization Tool is deprecated and will be removed in the future. Please use Neural Network Compression Framework instead: https://github.com/openvinotoolkit/nncf
Nevergrad package could not be imported. If you are planning to use any hyperparameter optimization algo, consider installing it using pip. This implies advanced usage of the tool. Note that nevergrad is compatible only with Python 3.7+


Generated JSON Output :  {'text_sequence': "Walaut's Save money. Live better. suvericy's Md Md M"}
elapsed time:  2.223904781000101
2023-07-22 12:24:33 | INFO     | Running Speedster on GPU:0


AttributeError: ignored